In [52]:
import panel as pn
import pandas as pd
import hvplot.pandas
import requests as r
from dotenv import load_dotenv
import os
from datetime import datetime as dt
import numpy as np

In [2]:
load_dotenv()

True

### Creating custom alpaca API request as the SDK has conflicting requiremnts with pyviz

In [32]:
def sharpe_ratio(df, trading_days, ):
    return np.sqrt()

In [37]:
def get_closing_prices(tickers, start_date=None, end_date=None):
    headers = {
        'APCA-API-KEY-ID':os.getenv('APCA_API_KEY_ID'),
        'APCA-API-SECRET-KEY':os.getenv('APCA_API_SECRET_KEY')
    }
    url = 'https://data.alpaca.markets/v1/bars/day?symbols=' + tickers
    if start_date:
        start_date = pd.Timestamp(start_date, tz='EST').isoformat()
        url += "&start=" + start_date
    if end_date:
        end_date = pd.Timestamp(end_date, tz='EST').isoformat()
        url += "&end=" + end_date
    ticker_data = r.get(url, headers=headers).json()
    
    # Extracting closing prices
    closing_prices = pd.DataFrame()
    closing_prices['date'] = [dt.fromtimestamp(x['t']) for x in ticker_data[tickers.split(',')[0]]]
    for k in ticker_data:
        closing_prices[k] = [x['c'] for x in ticker_data[k]]
        
    return closing_prices.set_index('date')

In [74]:
def graph_closes(tickers, start_date='2020-01-01', end_date='2020-02-01', calc='close'):
    try:
        df = get_closing_prices(tickers, start_date, end_date)
    except Exception as e:
        print(e)
        return None
    
    def plot_std(rolling, rolling_window):
        if rolling:
            return df.rolling(rolling_window).std().hvplot.line()
        else:
            return df.std().hvplot.bar()
        
    def plot_mean(rolling, rolling_window):
        if rolling:
            return df.rolling(rolling_window).mean().hvplot.line()
        else:
            return df.mean().hvplot.bar()
        
    def plot_ewma(rolling_window):
        return df.ewm(halflife=rolling_window).mean().hvplot.line()
    
    def plot_ewmstd(rolling_window):
        return df.ewm(halflife=rolling_window).std().hvplot.line()
    
    def plot_sharpe(rolling, rolling_window):
        returns = df.pct_change()
        if rolling:
            return returns.rolling(rolling_window).apply(lambda x: (x.mean() - 0.02) / x.std(), raw=True).hvplot.line()
        else:
            return ((returns.mean() * len(df) / (returns.std() * np.sqrt(len(df))))).hvplot.bar()
    
    if calc == 'close': return df.hvplot.line()
    if calc == 'pct_change': return df.pct_change().hvplot.line()
    if calc == 'ewma': return pn.interact(plot_ewma, rolling_window=(1, 20))
    if calc == 'ewmstd': return pn.interact(plot_ewmstd, rolling_window=(1, 20))
    if calc == 'std': return pn.interact(plot_std, rolling=False, rolling_window=(1, 20))
    if calc == 'mean': return pn.interact(plot_mean, rolling=False, rolling_window=(1, 20))
    if calc == 'sharpe': return pn.interact(plot_sharpe, rolling=False, rolling_window=(1, 20), num_trading_days=(1, 252))

In [75]:
pn.interact(graph_closes, 
            tickers='AAPL,TSLA', 
            start_date='2019-01-01', 
            end_date='2020-01-01', 
            calc=['close', 'pct_change', 'std', 'mean', 'ewma', 'ewmstd', 'sharpe'],
)

Column
    [0] Column
        [0] TextInput(name='tickers', value='AAPL,TSLA')
        [1] TextInput(name='start_date', value='2019-01-01')
        [2] TextInput(name='end_date', value='2020-01-01')
        [3] Select(name='calc', options=['close', 'pct_change', ...], value='close')
    [1] Row
        [0] HoloViews(NdOverlay, name='interactive285475')

In [59]:
((df.mean() * 252) / (df.std() * np.sqrt(252)))

AAPL    111.117633
dtype: float64

In [60]:
df

,AAPL
date,
2020-02-25 23:00:00,292.69
2020-02-26 23:00:00,273.04
2020-02-27 23:00:00,274.64
2020-03-01 23:00:00,298.82
2020-03-02 23:00:00,289.31
...,...
2020-07-12 23:00:00,381.91
2020-07-13 23:00:00,388.21
2020-07-14 23:00:00,390.91


In [73]:
((df.pct_change().mean() * len(df) / (df.pct_change().std() * np.sqrt(len(df)))))

AAPL    0.947014
dtype: float64